# Solución del péndulo invertido con control y observador calculado empleando el método de MH

El método MH está desarrollado en el artículo titulado "Controlador óptimo con observación de estados basado en el principio del máximo de Pontryagin de bajo costo computacional" [éste](https://riaa-tecno.unca.edu.ar/handle/123456789/1193). Disponible en éste [Repositorio](https://riaa-tecno.unca.edu.ar/bitstream/handle/123456789/1193/REVISTA%20CODINOA%20N%c2%ba%209_%20%28623-629%29%28PUCHETA%29.pdf?sequence=1&isAllowed=y).


*   Elemento de lista
*   Elemento de lista




In [ ]:
import Pkg; Pkg.add("Polynomials")

In [ ]:
Pkg.add("LaTeXStrings")

In [ ]:
using Polynomials
using LinearAlgebra
using Plots
using LaTeXStrings

In [ ]:
# Clear workspace equivalent (not needed in Julia scripts)
# Initialize parameters
m = 0.1
Fricc = 0.1
long = 0.6
g = 9.8
M = 0.5
h = 0.001
tiempo = Int(120 / h)
tita_pp = 0
t = 0:h:(tiempo * h)
# Initial conditions
omega = zeros(tiempo + 1)
p_p = zeros(tiempo + 1)
u = zeros(tiempo + 1)
p = zeros(tiempo + 1)
psi = zeros(tiempo + 1)
y_sal = zeros(tiempo + 1)
Jn = zeros(tiempo + 1)
i = 1
indice = 0
# Linearized system matrices (Sontag, p. 104)
# State = [p; p_p; alfa; omega]
Mat_A = [0 1 0 0;
         0 -Fricc/M -m*g/M 0;
         0 0 0 1;
         0 Fricc/(long*M) g*(m + M)/(long*M) 0]
Mat_B = [0; 1/M; 0; -1/(long*M)]
Mat_C=zeros((2, 4))
Mat_C = [1 0 0 0; 0 0 1 0]  # Output is position and angle
# Augmented system for integral control
Mat_Aa = [Mat_A zeros(4, 1); -Mat_C[1, :]' 0]
Mat_Ba = [Mat_B; 0]
# Controllability matrix
Mat_M = [Mat_Ba Mat_Aa*Mat_Ba Mat_Aa^2*Mat_Ba Mat_Aa^3*Mat_Ba Mat_Aa^4*Mat_Ba]
# Controller design by pole placement
auto_val = eigvals(Mat_Aa)
c_ai = fromroots(Polynomial, auto_val)
c_ai=c_ai[end:-1:1]
Mat_W = [c_ai[5] c_ai[4] c_ai[3] c_ai[2] 1;
         c_ai[4] c_ai[3] c_ai[2] 1 0;
         c_ai[3] c_ai[2] 1 0 0;
         c_ai[2] 1 0 0 0;
         1 0 0 0 0]
Mat_T = Mat_M * Mat_W
A_controlable = inv(Mat_T) * Mat_Aa * Mat_T

5×5 Matrix{Float64}:
 -1.74826e-15  1.0           8.94405e-20   2.71891e-18   9.72901e-20
  4.17625e-15  0.0           1.0           2.92741e-32   2.71891e-18
 -3.41061e-14  8.70052e-17   2.8064e-17    1.0          -3.96894e-20
  6.82121e-14  6.31089e-30  -7.88861e-31   1.47911e-31   1.0
 -6.68479e-13  1.64579e-14   3.26667      19.6          -0.2

In [ ]:
# Desired characteristic polynomial coefficients
#alfa_ia = [1.0, 10.724, 37.126, 48.193, 32.590, 10.330]
#pol = alfa_ia[2:end] - c_ai[2:end]
a5 = -A_controlable[5, 1]
a4 = -A_controlable[5, 2]
a3 = -A_controlable[5, 3]
a2 = -A_controlable[5, 4]
a1 = -A_controlable[5, 5]
# LQR controller parameters
R = 1e2
q5 = 1e5
q4 = 1e5
q3 = 1e5
q2 = 1e3
q1 = 1e1
Q = Diagonal([q1, q2, q3, q4, q5])
p1 = 0.5 * (-4*a5*R + sqrt((4*a5*R)^2 + 16*q1*R))
p2 = 0.5 * (-4*a4*R + sqrt((4*a4*R)^2 + 16*q2*R))
p3 = 0.5 * (-4*a3*R + sqrt((4*a3*R)^2 + 16*q3*R))
p4 = 0.5 * (-4*a2*R + sqrt((4*a2*R)^2 + 16*q4*R))
p5 = 0.5 * (-3*a1*R + sqrt((3*a1*R)^2 + 8*q5*R))
Ka = ([p1 p2 p3 p4 p5] / (2*R)) * inv(Mat_T)
K = Ka[1:4]
KI = -Ka[5]
println("Controller Poles:")
println(eigvals(Mat_Aa - Mat_Ba * Ka))

Controller Poles:
ComplexF64[-20.686647076049507 + 0.0im, -0.8129938196830493 - 0.8348550606202934im, -0.8129938196830493 + 0.8348550606202934im, -0.04927408460865383 - 0.0939639342910996im, -0.04927408460865383 + 0.0939639342910996im]


In [ ]:
# Observer design (LQR-based)
Mat_Adual = Mat_A'
Mat_Bdual = Mat_C'
Mat_Cdual = Mat_B'
Qdual = Diagonal([1, 1, 1, 10000])
Rdual = 0.01 * I(2)
Ha = [Mat_Adual -Mat_Bdual * inv(Rdual) * Mat_Bdual'; -Qdual -Mat_Adual']
n = size(Ha, 1)
n_a = size(Mat_A, 1)
D, V = eigen(Ha);
#show(stdout, "text/plain", V)
MX1X2 = zeros(ComplexF64,n, n_a) # Initialize MX1X2 as an empty matrix with the correct number of rows
for ii in 1:n
    if real(D[ii]) < 0
        MX1X2[:,ii] = V[:, ii]
    end
end
MX1 = MX1X2[1:n÷2, :]
MX2 = MX1X2[n÷2+1:end, :]
P = real(MX2 * inv(MX1))
Ko1 = (inv(Rdual) * Mat_Bdual' * P)'
println("LQR Observer Poles:")
#println(eigvals(Mat_A - Ko1 * Mat_C))
show(stdout, "text/plain", eigvals(Mat_A - Ko1 * Mat_C))

LQR Observer Poles:
4-element Vector{ComplexF64}:
  -23.12779588975478 - 21.570699182913735im
  -23.12779588975478 + 21.570699182913735im
  -9.947381050239205 + 0.0im
 -1.0439594266738634 + 0.0im

## Cálculo del Observador empleando el método de MH

In [ ]:
# Observer design (MH method)
h_ = 1e-9 * [-1; -1]
Mat_Bdual = Mat_C' * h_
Mat_Adual = Mat_A'
Qdual = Diagonal([1, 1000, 100000, 100])
Mat_M = [Mat_Bdual Mat_Adual*Mat_Bdual Mat_Adual^2*Mat_Bdual Mat_Adual^3*Mat_Bdual]
auto_val = eigvals(Mat_Adual)
c_ai = fromroots(Polynomial, auto_val)
c_ai=c_ai[end:-1:1]
Mat_W = [c_ai[4] c_ai[3] c_ai[2] 1;
         c_ai[3] c_ai[2] 1 0;
         c_ai[2] 1 0 0;
         1 0 0 0]
Mat_T = Mat_M * Mat_W
A_controlable = inv(Mat_T) * Mat_Adual * Mat_T
a4 = -A_controlable[4, 1]
a3 = -A_controlable[4, 2]
a2 = -A_controlable[4, 3]
a1 = -A_controlable[4, 4]
q4 = Qdual[4, 4]
q3 = Qdual[3, 3]
q2 = Qdual[2, 2]
q1 = Qdual[1, 1]
Ro = 0.1
p1 = 0.5 * (-4*a4*Ro + sqrt((4*a4*Ro)^2 + 16*q1*Ro))
p2 = 0.5 * (-4*a3*Ro + sqrt((4*a3*Ro)^2 + 16*q2*Ro))
p3 = 0.5 * (-4*a2*Ro + sqrt((4*a2*Ro)^2 + 16*q3*Ro))
p4 = 0.5 * (-3*a1*Ro + sqrt((3*a1*Ro)^2 + 8*q4*Ro))
ko = ([p1 p2 p3 p4] / (2*Ro)) * inv(Mat_T)
Ko = (h_ * ko)'
println("MH Observer Poles:")
#println(eigvals(Mat_A - Ko * Mat_C))
show(stdout, "text/plain", eigvals(Mat_A - Ko * Mat_C))

MH Observer Poles:
4-element Vector{ComplexF64}:
   -11.15549760906352 - 29.555176861927833im
   -11.15549760906352 + 29.555176861927833im
 -0.05009383325293949 - 0.025678202143909763im
 -0.05009383325293949 + 0.025678202143909763im

## Simulación del controlador con el observador, desde cualquier condición inicial para el ángulo $\phi_0$ [$-\pi, \pi$].

In [ ]:
# Simulation
ref = 100
T_CI = 2
Mat_datop = zeros(T_CI, tiempo + 1)
Mat_datop_p = zeros(T_CI, tiempo + 1)
Mat_datoalfa = zeros(T_CI, tiempo + 1)
Mat_datoomega = zeros(T_CI, tiempo + 1)
Mat_datou = zeros(T_CI, tiempo + 1)
TamanioFuente = 12
Ts = h
for ci in 1:T_CI
    x_hat = zeros(4)
    Jn[1] = 0
    omega[1] = 0
    p_p[1] = 0
    u[1] = 0
    p[1] = 0
    i = 1
    alfa = zeros(tiempo + 1)
    if ci == 1
        alfa[1] = -pi
        color = :red
    elseif ci == 2
        alfa[1] = 0.6
        color = :red
    elseif ci == 3
        alfa[1] = -0.6
        color = :green
    elseif ci == 4
        alfa[1] = pi
        color = :blue
    end
    Mat_datop[ci, 1] = p[1]
    Mat_datop_p[ci, 1] = p_p[1]
    Mat_datoalfa[ci, 1] = alfa[1]
    Mat_datoomega[ci, 1] = omega[1]
    Mat_datou[ci, 1] = u[1]
    while i < tiempo + 1
        estado = [p[i]; p_p[i]; alfa[i]; omega[i]]
        psi_p = ref - Mat_C[1, :]' * estado
        psi[i+1] = psi[i] + psi_p * h
        u[i] = -K' * x_hat + KI * psi[i+1]
        p_pp = (1/(M + m)) * (u[i] - m*long*tita_pp*cos(alfa[i]) + m*long*omega[i]^2*sin(alfa[i]) - Fricc*p_p[i])
        tita_pp = (1/long) * (g*sin(alfa[i]) - p_pp*cos(alfa[i]))
        p_p[i+1] = p_p[i] + h * p_pp
        p[i+1] = p[i] + h * p_p[i]
        omega[i+1] = omega[i] + h * tita_pp
        alfa[i+1] = alfa[i] + h * omega[i]
        Y_ = Mat_C * estado
        y_sal[i] = Y_[1]
        Jn[i+1] = Jn[i] + estado' * Q[1:4, 1:4] * estado + u[i] * R * u[i]
        # Observer
        y_sal_O = Mat_C * x_hat
        x_hatp = Mat_A * x_hat + Mat_B * u[i] + Ko * (Y_ - y_sal_O)
        x_hat = x_hat + h * x_hatp
        i += 1
    end
    u[i] = -K' * x_hat + KI * psi[i]
    Mat_datop[ci, :] = p
    Mat_datop_p[ci, :] = p_p
    Mat_datoalfa[ci, :] = alfa
    Mat_datoomega[ci, :] = omega
    Mat_datou[ci, :] = u
    # Plotting (Phase plots)
    p1 = plot(alfa, omega, color=color, label="", title="Ángulo", xlabel=L"\phi_t", ylabel=L"\dot{\phi}_t", grid=true, titlefontsize=TamanioFuente, guidefontsize=TamanioFuente)
    p2 = plot(p, p_p, color=color, label="", title="Desplazamiento de carro", xlabel="δ_t", ylabel=L"\dot{\delta}_t", grid=true, titlefontsize=TamanioFuente, guidefontsize=TamanioFuente)
    p3 = plot(t, log10.(Jn), color=color, label="", title="Modelo no lineal", xlabel="Tiempo en Seg.", ylabel="J(x,u)", grid=true, titlefontsize=TamanioFuente, guidefontsize=TamanioFuente)
    plot(p1, p2, p3, layout=(3, 2))
    display(plot!())
end

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Time-domain plots
p1 = plot(t, Mat_datoalfa', title=L"\phi_t", grid=true, label="", titlefontsize=TamanioFuente)
p2 = plot(t, Mat_datoomega', title=L"\dot{\phi}_t", grid=true, label="", titlefontsize=TamanioFuente)
p3 = plot(t, Mat_datop', title=L"\delta_t", grid=true, label="", titlefontsize=TamanioFuente)
p4 = plot(t, Mat_datop_p', title=L"\dot{\delta}_t", grid=true, label="", titlefontsize=TamanioFuente)
p5 = plot(t, Mat_datou', title="Acción de control", xlabel="Tiempo en Seg.", grid=true, label="", titlefontsize=TamanioFuente)
plot(p1, p2, p3, p4, p5, layout=(3, 2))
display(plot!())

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
plot(p1, p2, p3, p4, p5, layout=[grid(2,2); (2,1)])
display(plot!())

Output hidden; open in https://colab.research.google.com to view.

# Conclusiones

Se ha mostrado el desempeño de un controlador con observación de estados desarrollado por investigadores que trabajan en la temática.